In [ ]:
!pip install pycountry

     |████████████████████████████████| 10.1MB 2.7MB/s 
  Created wheel for pycountry: filename=pycountry-20.7.3-py2.py3-none-any.whl size=10746865 sha256=6b39e07477077e19f9432a6c0f346801ad320cc0f09cd3f0c85d500dd6dd412a
  Stored in directory: /root/.cache/pip/wheels/33/4e/a6/be297e6b83567e537bed9df4a93f8590ec01c1acfbcd405348
Successfully built pycountry


Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import plotly.graph_objects as go
from fbprophet import Prophet
import pycountry
import plotly.express as px
from collections import namedtuple

Data Import, Preprocessing and EDA

In [ ]:
df = pd.read_csv('/content/covid_19_data.csv',parse_dates=['Last Update'])
df.rename(columns={'ObservationDate':'Date', 'Country/Region':'Country'}, inplace=True)
#these are time series datasets=> updated on a daily basis (show the dataset)
#dataset of confirmed cases
df_confirmed = pd.read_csv("/content/time_series_covid_19_confirmed.csv")
#dataset of recovered cases
df_recovered = pd.read_csv("/content/time_series_covid_19_recovered.csv")
#dataset of deaths
df_deaths = pd.read_csv("/content/time_series_covid_19_deaths.csv")

df_confirmed.rename(columns={'Country/Region':'Country'}, inplace=True)
df_recovered.rename(columns={'Country/Region':'Country'}, inplace=True)
df_deaths.rename(columns={'Country/Region':'Country'}, inplace=True)


In [ ]:
df_confirmed.head()

,Province/State,Country,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,...,8/15/20,8/16/20,8/17/20,8/18/20,8/19/20,8/20/20,8/21/20,8/22/20,8/23/20,8/24/20,8/25/20,8/26/20,8/27/20,8/28/20,8/29/20,8/30/20,8/31/20,9/1/20,9/2/20,9/3/20,9/4/20,9/5/20,9/6/20,9/7/20,9/8/20,9/9/20,9/10/20,9/11/20,9/12/20,9/13/20,9/14/20,9/15/20,9/16/20,9/17/20,9/18/20,9/19/20,9/20/20,9/21/20,9/22/20,9/23/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,...,37551,37596,37599,37599,37599,37856,37894,37953,37999,38054,38070,38113,38129,38140,38143,38162,38165,38196,38243,38288,38304,38324,38398,38494,38520,38544,38572,38606,38641,38716,38772,38815,38855,38872,38883,38919,39044,39074,39096,39145
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,7260,7380,7499,7654,7812,7967,8119,8275,8427,8605,8759,8927,9083,9195,9279,9380,9513,9606,9728,9844,9967,10102,10255,10406,10553,10704,10860,11021,11185,11353,11520,11672,11816,11948,12073,12226,12385,12535,12666,12787
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,...,38133,38583,39025,39444,39847,40258,40667,41068,41460,41858,42228,42619,43016,43403,43781,44146,44494,44833,45158,45469,45773,46071,46364,46653,46938,47216,47488,47752,48007,48254,48496,48734,48966,49194,49413,49623,49826,50023,50214,50400
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,989,989,1005,1005,1024,1024,1045,1045,1045,1060,1060,1098,1098,1124,1124,1124,1176,1184,1199,1199,1215,1215,1215,1261,1261,1301,1301,1344,1344,1344,1438,1438,1483,1483,1564,1564,1564,1681,1681,1753
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1879,1906,1935,1966,2015,2044,2068,2134,2171,2222,2283,2332,2415,2471,2551,2624,2654,2729,2777,2805,2876,2935,2965,2981,3033,3092,3217,3279,3335,3388,3439,3569,3675,3789,3848,3901,3991,4117,4236,4363


Country wise confirmed, recovered and death cases

In [ ]:
#country wise 
df2 = df.groupby(["Date", "Country", "Province/State"])[['SNo', 'Date', 'Province/State', 'Country', 'Confirmed', 'Deaths', 'Recovered']].sum().reset_index()
df2

,Date,Country,Province/State,SNo,Confirmed,Deaths,Recovered
0,01/22/2020,Hong Kong,Hong Kong,13,0.0,0.0,0.0
1,01/22/2020,Macau,Macau,21,1.0,0.0,0.0
2,01/22/2020,Mainland China,Anhui,1,1.0,0.0,0.0
3,01/22/2020,Mainland China,Beijing,2,14.0,0.0,0.0
4,01/22/2020,Mainland China,Chongqing,3,6.0,0.0,0.0
...,...,...,...,...,...,...,...
81443,09/23/2020,Ukraine,Vinnytsia Oblast,116773,5455.0,103.0,3580.0
81444,09/23/2020,Ukraine,Volyn Oblast,116779,7132.0,150.0,4931.0
81445,09/23/2020,Ukraine,Zakarpattia Oblast,116800,9343.0,294.0,4295.0
81446,09/23/2020,Ukraine,Zaporizhia Oblast,116801,3149.0,49.0,1158.0


For India

In [ ]:
df.query('Country=="India"').groupby("Last Update")[['Confirmed', 'Deaths', 'Recovered']].sum().reset_index()

,Last Update,Confirmed,Deaths,Recovered
0,2020-01-30 16:00:00,1.0,0.0,0.0
1,2020-01-31 08:15:00,1.0,0.0,0.0
2,2020-01-31 23:59:00,1.0,0.0,0.0
3,2020-02-02 06:03:08,2.0,0.0,0.0
4,2020-02-03 21:43:02,30.0,0.0,0.0
...,...,...,...,...
204,2020-09-20 04:22:56,5308014.0,85619.0,4208431.0
205,2020-09-21 04:23:18,5400619.0,86752.0,4303043.0
206,2020-09-22 04:23:11,5487580.0,87882.0,4396399.0
207,2020-09-23 04:23:40,5562663.0,88935.0,4497867.0


Countries sorted alphabetically


In [ ]:
df.groupby("Country")[['Confirmed', 'Deaths', 'Recovered']].sum().reset_index()

,Country,Confirmed,Deaths,Recovered
0,Azerbaijan,1.0,0.0,0.0
1,"('St. Martin',)",2.0,0.0,0.0
2,Afghanistan,4134817.0,128968.0,2463817.0
3,Albania,704767.0,20902.0,398544.0
4,Algeria,3583765.0,162118.0,2438290.0
...,...,...,...,...
218,Western Sahara,1481.0,121.0,1112.0
219,Yemen,177772.0,49501.0,86008.0
220,Zambia,750629.0,18002.0,635259.0
221,Zimbabwe,394970.0,10243.0,240370.0


Date wise number of cases(Worldwide)

In [ ]:
df.groupby('Date').sum()

,SNo,Confirmed,Deaths,Recovered
Date,,,,
01/22/2020,741,555.0,17.0,28.0
01/23/2020,2829,653.0,18.0,30.0
01/24/2020,4305,941.0,26.0,36.0
01/25/2020,6490,1438.0,42.0,39.0
01/26/2020,9071,2118.0,56.0,52.0
...,...,...,...,...
09/19/2020,84189917,30688150.0,955866.0,20922189.0
09/20/2020,84740481,30935011.0,959565.0,21159459.0
09/21/2020,85406364,31245797.0,963693.0,21394593.0


## Visualizations

We have used plotly to make interactive visualizations

Bar Chart to summarize Worldwide Cases - Confirmed, Deaths & Recovered

In [ ]:
confirmed = df.groupby('Date').sum()['Confirmed'].reset_index()
deaths = df.groupby('Date').sum()['Deaths'].reset_index()
recovered = df.groupby('Date').sum()['Recovered'].reset_index()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x=confirmed['Date'],
                y=confirmed['Confirmed'],
                name='Confirmed',
                marker_color='blue'
                ))
fig.add_trace(go.Bar(x=deaths['Date'],
                y=deaths['Deaths'],
                name='Deaths',
                marker_color='Red'
                ))
fig.add_trace(go.Bar(x=recovered['Date'],
                y=recovered['Recovered'],
                name='Recovered',
                marker_color='Green'
                ))

fig.update_layout(
    title='Worldwide Corona Virus Cases - Confirmed, Deaths, Recovered (Bar Chart)',
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='Number of Cases',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.15, # gap between bars of adjacent location coordinates.
    bargroupgap=0.1 # gap between bars of the same location coordinate.
)
fig.show()

Line Chart to summarize Worldwide Cases - Confirmed, Deaths & Recovered

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=confirmed['Date'], 
                         y=confirmed['Confirmed'],
                         mode='lines+markers',
                         name='Confirmed',
                         line=dict(color='blue', width=2)
                        ))
fig.add_trace(go.Scatter(x=deaths['Date'], 
                         y=deaths['Deaths'],
                         mode='lines+markers',
                         name='Deaths',
                         line=dict(color='Red', width=2)
                        ))
fig.add_trace(go.Scatter(x=recovered['Date'], 
                         y=recovered['Recovered'],
                         mode='lines+markers',
                         name='Recovered',
                         line=dict(color='Green', width=2)
                        ))
fig.update_layout(
    title='Worldwide Corona Virus Cases - Confirmed, Deaths, Recovered (Line Chart)',
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='Number of Cases',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    )
)
fig.show()

Time lapse plot (main reason to use plotly)

In [ ]:
df_confirmed = df_confirmed[["Province/State","Lat","Long","Country"]]
df_temp = df.copy()
#replace mainland china to china
df_temp['Country'].replace({'Mainland China': 'China'}, inplace=True)
df_latlong = pd.merge(df_temp, df_confirmed, on=["Country", "Province/State"])

In [ ]:
fig = px.density_mapbox(df_latlong, 
                        lat="Lat", 
                        lon="Long", 
                        hover_name="Province/State", 
                        hover_data=["Confirmed","Deaths","Recovered"], 
                        animation_frame="Date",
                        color_continuous_scale="Portland",
                        radius=7, 
                        zoom=0,height=700)
fig.update_layout(title='Worldwide Corona Virus Cases Time Lapse - Confirmed, Deaths, Recovered',
                  font=dict(family="Courier New, monospace",
                            size=18,
                            color="#7f7f7f")
                 )
fig.update_layout(mapbox_style="open-street-map", mapbox_center_lon=0)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})


fig.show()

### Analysis by Country

Latest entry in the Dataset

In [ ]:
#df2->country wise data
confirmed = df2.groupby(['Date', 'Country']).sum()[['Confirmed']].reset_index()
deaths = df2.groupby(['Date', 'Country']).sum()[['Deaths']].reset_index()
recovered = df2.groupby(['Date', 'Country']).sum()[['Recovered']].reset_index()

In [ ]:
latest_date = confirmed['Date'].max()
latest_date

'09/23/2020'

Latest cases according to the dataset

In [ ]:
confirmed = confirmed[(confirmed['Date']==latest_date)][['Country', 'Confirmed']]
deaths = deaths[(deaths['Date']==latest_date)][['Country', 'Deaths']]
recovered = recovered[(recovered['Date']==latest_date)][['Country', 'Recovered']]

In [ ]:
confirmed

,Country,Confirmed
4023,Australia,26980.0
4024,Brazil,4591364.0
4025,Canada,149939.0
4026,Chile,449903.0
4027,Colombia,784268.0
4028,Denmark,465.0
4029,France,24500.0
4030,Germany,279025.0
4031,Hong Kong,5049.0
4032,India,5646010.0


### Bubble Plot construction

For interactive bubble plot we needed to do some processing to the country names, as some countries are found to be different in pycountry.countries 
(Eg: such as Mainland China instead of China)

In [ ]:
#list of countries in pycountry.countries
print(list(country.name for country in pycountry.countries))

['Aruba', 'Afghanistan', 'Angola', 'Anguilla', 'Åland Islands', 'Albania', 'Andorra', 'United Arab Emirates', 'Argentina', 'Armenia', 'American Samoa', 'Antarctica', 'French Southern Territories', 'Antigua and Barbuda', 'Australia', 'Austria', 'Azerbaijan', 'Burundi', 'Belgium', 'Benin', 'Bonaire, Sint Eustatius and Saba', 'Burkina Faso', 'Bangladesh', 'Bulgaria', 'Bahrain', 'Bahamas', 'Bosnia and Herzegovina', 'Saint Barthélemy', 'Belarus', 'Belize', 'Bermuda', 'Bolivia, Plurinational State of', 'Brazil', 'Barbados', 'Brunei Darussalam', 'Bhutan', 'Bouvet Island', 'Botswana', 'Central African Republic', 'Canada', 'Cocos (Keeling) Islands', 'Switzerland', 'Chile', 'China', "Côte d'Ivoire", 'Cameroon', 'Congo, The Democratic Republic of the', 'Congo', 'Cook Islands', 'Colombia', 'Comoros', 'Cabo Verde', 'Costa Rica', 'Cuba', 'Curaçao', 'Christmas Island', 'Cayman Islands', 'Cyprus', 'Czechia', 'Germany', 'Djibouti', 'Dominica', 'Denmark', 'Dominican Republic', 'Algeria', 'Ecuador', 'Egy

In [ ]:
#making copy of the country wise data for the plot
confirmed2 = confirmed.copy()
deaths2 = deaths.copy()
recovered2 = recovered.copy()
bubble_plot_dfs = [confirmed2, deaths2, recovered2]
for df_ in bubble_plot_dfs:
    df_["Country"].replace({'Mainland China': 'China'}, inplace=True)
    df_["Country"].replace({'UK': 'United Kingdom'}, inplace=True)
    df_["Country"].replace({'US': 'United States'}, inplace=True)

In [ ]:
countries = {}
for country in pycountry.countries:
    countries[country.name] = country.alpha_3
    
confirmed2["iso_alpha"] = confirmed2["Country"].map(countries.get)
deaths2["iso_alpha"] = deaths2["Country"].map(countries.get)
recovered2["iso_alpha"] = recovered2["Country"].map(countries.get)

#iso_alpha => country codes Eg: India => IND

In [ ]:
confirmed2

,Country,Confirmed,iso_alpha
4023,Australia,26980.0,AUS
4024,Brazil,4591364.0,BRA
4025,Canada,149939.0,CAN
4026,Chile,449903.0,CHL
4027,Colombia,784268.0,COL
4028,Denmark,465.0,DNK
4029,France,24500.0,FRA
4030,Germany,279025.0,DEU
4031,Hong Kong,5049.0,HKG
4032,India,5646010.0,IND


In [ ]:
plot_data_confirmed = confirmed2[["iso_alpha","Confirmed", "Country"]]
plot_data_deaths = deaths2[["iso_alpha","Deaths"]]
plot_data_recovered = recovered2[["iso_alpha","Recovered"]]

Plot for confirmed cases:

In [ ]:
fig = px.scatter_geo(plot_data_confirmed, locations="iso_alpha", color="Country",
                     hover_name="iso_alpha", size="Confirmed",
                     projection="natural earth", title = 'Worldwide Confirmed Cases')
fig.show()

Plot for death cases:

In [ ]:
fig = px.scatter_geo(plot_data_deaths, locations="iso_alpha", color="Deaths",
                     hover_name="iso_alpha", size="Deaths",
                     projection="natural earth", title="Worldwide Death Cases")
fig.show()

Plot for recovered cases:

In [ ]:
fig = px.scatter_geo(plot_data_recovered, locations="iso_alpha", color="Recovered",
                     hover_name="iso_alpha", size="Recovered",
                     projection="natural earth", title="Worldwide Recovered Cases")
fig.show()

## Forecasting Total Number of Cases Worldwide

Transforming data for prophet

In [ ]:
confirmed = df.groupby('Date').sum()['Confirmed'].reset_index()
deaths = df.groupby('Date').sum()['Deaths'].reset_index()
recovered = df.groupby('Date').sum()['Recovered'].reset_index()
confirmed

,Date,Confirmed
0,01/22/2020,555.0
1,01/23/2020,653.0
2,01/24/2020,941.0
3,01/25/2020,1438.0
4,01/26/2020,2118.0
...,...,...
241,09/19/2020,30688150.0
242,09/20/2020,30935011.0
243,09/21/2020,31245797.0
244,09/22/2020,31517087.0


In [ ]:
confirmed.columns = ['ds','y']
confirmed['ds'] = pd.to_datetime(confirmed['ds'])
confirmed.tail()   

,ds,y
241,2020-09-19,30688150.0
242,2020-09-20,30935011.0
243,2020-09-21,31245797.0
244,2020-09-22,31517087.0
245,2020-09-23,31779835.0


Forecasting Confirmed Cases Worldwide with Prophet

In [ ]:
m = Prophet(interval_width=0.95)
m.fit(confirmed)
#making pred for next 30 days
future = m.make_future_dataframe(periods=30)
future_confirmed = future.copy() # for predictions later on
future.tail()

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


,ds
271,2020-10-19
272,2020-10-20
273,2020-10-21
274,2020-10-22
275,2020-10-23


In [ ]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
271,2020-10-19,3.841249e+07,3.753680e+07,3.920295e+07
272,2020-10-20,3.867382e+07,3.775156e+07,3.948939e+07
273,2020-10-21,3.894282e+07,3.795672e+07,3.983736e+07
274,2020-10-22,3.920872e+07,3.820312e+07,4.014667e+07
275,2020-10-23,3.948552e+07,3.842839e+07,4.042941e+07


**RMSE**

In [ ]:
from sklearn.metrics import mean_squared_error
true_pred_df = pd.merge(forecast[['ds', 'yhat']], confirmed, on='ds')[['y','yhat']]
mean_squared_error(true_pred_df['y'], true_pred_df['yhat'], squared=False)

37359.72047579066

In [ ]:
from fbprophet.plot import plot_plotly, plot_components_plotly
confirmed_forecast_plot = plot_plotly(m, forecast)
confirmed_forecast_plot

In [ ]:
forecast_components = plot_components_plotly(m, forecast)
forecast_components

Forecast component plots show an upward trend in the number of cases worldwide. 
In the weekly trends plot, it shows the highest no of cases at the weekends.

Forecasting worldwide deaths with prophet

In [ ]:
#30 days ahead forecast with 95% prediction intervals
deaths.columns = ['ds','y']
deaths['ds'] = pd.to_datetime(deaths['ds'])

In [ ]:
m = Prophet(interval_width=0.95)
m.fit(deaths)
future = m.make_future_dataframe(periods=30)
future_deaths = future.copy() # for non-baseline predictions later on
future.tail()

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


,ds
271,2020-10-19
272,2020-10-20
273,2020-10-21
274,2020-10-22
275,2020-10-23


In [ ]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
271,2020-10-19,1.118958e+06,1.080691e+06,1.159950e+06
272,2020-10-20,1.124964e+06,1.084431e+06,1.168108e+06
273,2020-10-21,1.131044e+06,1.087671e+06,1.176378e+06
274,2020-10-22,1.137123e+06,1.093208e+06,1.185380e+06
275,2020-10-23,1.143003e+06,1.096628e+06,1.194281e+06


In [ ]:
true_pred_df = pd.merge(forecast[['ds', 'yhat']], confirmed, on='ds')[['y','yhat']]
mean_squared_error(true_pred_df['y'], true_pred_df['yhat'], squared=False)

12714830.652547877

In [ ]:
deaths_forecast_plot = plot_plotly(m, forecast)
deaths_forecast_plot

In [ ]:
forecast_components = plot_components_plotly(m, forecast)
forecast_components

Forecasting Recovered Cases Worldwide with Prophet

In [ ]:
#same properties
recovered.columns = ['ds','y']
recovered['ds'] = pd.to_datetime(recovered['ds'])

In [ ]:
m = Prophet(interval_width=0.95)
m.fit(recovered)
future = m.make_future_dataframe(periods=30)
future_recovered = future.copy() # for non-baseline predictions later on
future.tail()

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


,ds
271,2020-10-19
272,2020-10-20
273,2020-10-21
274,2020-10-22
275,2020-10-23


In [ ]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
271,2020-10-19,2.730876e+07,2.666995e+07,2.800769e+07
272,2020-10-20,2.752712e+07,2.684754e+07,2.827771e+07
273,2020-10-21,2.774927e+07,2.703455e+07,2.851894e+07
274,2020-10-22,2.795790e+07,2.720285e+07,2.875389e+07
275,2020-10-23,2.817733e+07,2.737063e+07,2.902577e+07


In [ ]:
true_pred_df = pd.merge(forecast[['ds', 'yhat']], confirmed, on='ds')[['y','yhat']]
mean_squared_error(true_pred_df['y'], true_pred_df['yhat'], squared=False)

4961136.462672311

In [ ]:
recovered_forecast_plot = plot_plotly(m,forecast)
recovered_forecast_plot

In [ ]:
forecast_components = plot_components_plotly(m,forecast)
forecast_components